Recoherence

In [21]:
import json
import pennylane as qml
import pennylane.numpy as np

In [22]:
dev = qml.device("default.qubit", wires=5)


@qml.qnode(dev)
def evolve_state(coeffs, time):
    """
    Args:
        coeffs (list(float)): A list of the coupling constants g_1, g_2, g_3, and g_4
        time (float): The evolution time of th system under the given Hamiltonian

    Returns:
        (numpy.tensor): The density matrix for the evolved state of the central spin.
    """

    # We build the Hamiltonian for you

    operators = [
        qml.PauliZ(0) @ qml.PauliZ(1),
        qml.PauliZ(0) @ qml.PauliZ(2),
        qml.PauliZ(0) @ qml.PauliZ(3),
        qml.PauliZ(0) @ qml.PauliZ(4),
    ]
    hamiltonian = qml.Dot(coeffs, operators)

    # Apply the time evolution
    qml.Exp(hamiltonian * time)

    # Return the density matrix
    return qml.density_matrix(wires=0)

In [23]:
def purity(rho):
    """
    Args:
        rho (array(array(complex))): An array-like object representing a density matrix

    Returns:
        (float): The purity of the density matrix rho

    """

    return np.trace(np.dot(rho, rho)).real

In [24]:
def recoherence_time(coeffs):
    """
    Args:
        coeffs (list(float)): A list of the coupling constants g_1, g_2, g_3, and g_4.

    Returns:
        (float): The recoherence time of the central spin.

    """

    # Initial state parameters
    alpha_values = [np.pi / 2, 0.4, 1.2, 1.8, 0.6]

    # Calculate the initial density matrix
    initial_state = np.zeros((2, 2), dtype=np.complex128)
    for i, alpha in enumerate(alpha_values):
        state_i = np.array([np.cos(alpha / 2), np.sin(alpha / 2)], dtype=np.complex128)
        initial_state += np.outer(state_i, state_i.conj())

    # Set the initial state of the central spin
    dev.reset()
    dev._state = np.kron(initial_state, np.eye(2 ** 4))  # Set the initial state for the central spin

    # Set a small time step to search for the recoherence time
    dt = 0.01
    current_time = 0.0

    # Evolve the state and check the purity at each time step
    while True:
        state = evolve_state(coeffs, current_time)
        current_purity = purity(state)
        if np.isclose(current_purity, 1.0, rtol=5e-2):
            return current_time
        current_time += dt

In [25]:
# These functions are responsible for testing the solution.
def run(test_case_input: str) -> str:
    params = json.loads(test_case_input)
    output = recoherence_time(params)

    return str(output)


def check(solution_output: str, expected_output: str) -> None:
    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)

    assert np.isclose(solution_output, expected_output, rtol=5e-2)

# These are the public test cases
test_cases = [
    ('[5,5,5,5]', '0.314'),
    ('[1.1,1.3,1,2.3]', '15.71')
]

# This will run the public test cases locally
for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input '[5,5,5,5]'...
Runtime Error. 'DefaultQubit' object has no attribute 'reset'
Running test case 1 with input '[1.1,1.3,1,2.3]'...
Runtime Error. 'DefaultQubit' object has no attribute 'reset'
